# Get the data

Depending on each case your data may look a little bit different but we should start our portfolio optimization by acquiring information on a set of STOCK to better understand their behaviour trough, in this case, one year of activity in the market.

In [1]:
import pandas as pd

data = pd.read_csv("../data/binance_data.csv")
data

,Unnamed: 0,Asset,Open time,Open,High,Low,Close,Volume,Closing time,Quote asset vol,Num traders,Taker buy base asset vol,Taker buy quote asset vol,To be ignored
0,0,ETHBTC,1634774400000,0.063057,0.066299,0.062904,0.065148,219179.4803,1634860799999,14185.381377,441192,100293.9821,6492.487566,0
1,1,ETHBTC,1634860800000,0.065160,0.065960,0.064685,0.065446,97034.6275,1634947199999,6338.327303,199872,52745.0758,3446.038753,0
2,2,ETHBTC,1634947200000,0.065445,0.068079,0.065181,0.067997,78071.8103,1635033599999,5183.565102,170954,39062.4171,2593.452923,0
3,3,ETHBTC,1635033600000,0.067998,0.068163,0.066101,0.067079,75865.3395,1635119999999,5087.916649,161735,36494.9300,2447.367339,0
4,4,ETHBTC,1635120000000,0.067081,0.067245,0.065269,0.066897,83641.3418,1635206399999,5546.270472,180063,41199.6332,2731.174031,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1825,361,QTUMETH,1665964800000,0.002043,0.002058,0.002038,0.002053,1748.5000,1666051199999,3.577257,55,629.4000,1.289472,0
1826,362,QTUMETH,1666051200000,0.002053,0.002059,0.002020,0.002054,2889.4000,1666137599999,5.884752,147,795.5000,1.625642,0
1827,363,QTUMETH,1666137600000,0.002052,0.002063,0.002005,0.002011,3764.5000,1666223999999,7.690868,128,1750.6000,3.577481,0
1828,364,QTUMETH,1666224000000,0.002017,0.002057,0.002013,0.002030,3651.6000,1666310399999,7.457298,152,1642.9000,3.351432,0


Following this we will calculate the average expected revenue for each asset. This is done by getting the difference between *opening* and *closing* time scaled to the opening price. This way we make sure each asset is independently evaluated.

We will also compute the covariance between each asset so that we can consider this values as part of our portfolio diversification constraint.

In [2]:
from cmath import exp
import numpy as np

# Unique asset list
asset_list = data["Asset"].unique()

exp_ret = {}
return_list = []
for asset in asset_list:
    open_price = np.array(data[data["Asset"] == asset]["Open"].astype("float"))
    close_price = np.array(data[data["Asset"] == asset]["Close"].astype("float"))
        
    # Sign will be used to indicate the value gradient direction
    returns = ((close_price - open_price)/open_price)
    exp_ret[asset] = returns.mean()
    return_list.append(returns)

# Expected return on each asset
mu = [i for i in exp_ret.values()]   
    
# Compute covariance between returns
sigma = np.cov(np.vstack(return_list))

Here $\mu$ is the value associated with the expected average return for each asset.

In [3]:
for i,v in zip(asset_list, mu):
    print(f"Expected average return for asset {i} is {v}")

Expected average return for asset ETHBTC is 0.00043221237788004834
Expected average return for asset LTCBTC is -8.478051504195303e-05
Expected average return for asset BNBBTC is 0.0019369845670104667
Expected average return for asset NEOBTC is -0.0010735484309001167
Expected average return for asset QTUMETH is -0.0011878855362685353


And $\sigma$ is the covariance between those very same assets.

In [4]:
sigma

array([[ 0.00048188,  0.00022357,  0.00019375,  0.00021957, -0.00019389],
       [ 0.00022357,  0.00075058,  0.00022139,  0.00035433,  0.00022951],
       [ 0.00019375,  0.00022139,  0.00050048,  0.00023024,  0.00012029],
       [ 0.00021957,  0.00035433,  0.00023024,  0.00098326,  0.00046827],
       [-0.00019389,  0.00022951,  0.00012029,  0.00046827,  0.0010795 ]])

It is important to know what the cost is of each asset so that we can also limit the budget we would like to spend in our investment.

In [5]:
filter = data.groupby("Asset").agg({"Open time":max}).reset_index()
costs = data.merge(filter, how='inner').drop_duplicates()
cost_list = costs[["Asset","Open"]].to_dict('records')

We will store this information so that it can be used later.

In [6]:
import json

# Serializing json  
data = {"mu" : mu, "sigma": sigma.tolist(), "assets": cost_list} 
json_object = json.dumps(data, indent = 4)

with open("../data/portfolio-binance-data.json", "w") as file:
    file.write(json_object)